In [1]:
import os
os.chdir("/work/agoindan/.cache")
os.environ['TRANSFORMERS_CACHE'] = "/work/agoindan/.cache"
os.environ['TORCH_HUB'] = "/work/agoindan/.cache"
os.environ['XDG_CACHE_HOME'] = "/work/agoindan/.cache"

In [2]:
import sys
sys.path.append("/home/agoindan/HEMM")

In [3]:
import numpy as np

In [4]:
import autokeras as ak
from tensorflow.keras.models import load_model

In [5]:
cache_folder = "/work/agoindan/.cache/NSFW-cache"
clip_model = "ViT-L/14"

In [6]:
model_dir = cache_folder + "/clip_autokeras_binary_nsfw"
dim = 768

In [7]:
if not os.path.exists(model_dir):
    os.makedirs(cache_folder, exist_ok=True)

    from urllib.request import urlretrieve  # pylint: disable=import-outside-toplevel

    path_to_zip_file = cache_folder + "/clip_autokeras_binary_nsfw.zip"
    if clip_model == "ViT-L/14":
        url_model = "https://raw.githubusercontent.com/LAION-AI/CLIP-based-NSFW-Detector/main/clip_autokeras_binary_nsfw.zip"
    elif clip_model == "ViT-B/32":
        url_model = (
            "https://raw.githubusercontent.com/LAION-AI/CLIP-based-NSFW-Detector/main/clip_autokeras_nsfw_b32.zip"
        )
    else:
        raise ValueError("Unknown model {}".format(clip_model))  # pylint: disable=consider-using-f-string
    urlretrieve(url_model, path_to_zip_file)
    import zipfile  # pylint: disable=import-outside-toplevel

    with zipfile.ZipFile(path_to_zip_file, "r") as zip_ref:
        zip_ref.extractall(cache_folder)

loaded_model = load_model(model_dir, custom_objects=ak.CUSTOM_OBJECTS)

2024-02-23 14:32:22.387089: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-23 14:32:22.478351: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-23 14:32:22.480118: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-23 14:32:22.482492: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [19]:
import torch 
import clip 
from PIL import Image
import pickle 
from tqdm import tqdm

In [9]:
data = pickle.load(open("/work/agoindan/.cache/emu_magic_brush.pkl", "rb"))

In [13]:
def normalized(a, axis=-1, order=2):
    import numpy as np  # pylint: disable=import-outside-toplevel

    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2 == 0] = 1
    return a / np.expand_dims(l2, axis)

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14", device=device)

100%|███████████████████████████████████████| 933M/933M [08:01<00:00, 1.94MiB/s]


In [37]:
toxicity = []
for pred in tqdm(data["predictions"], total=len(data["predictions"])):
    img = preprocess(pred).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(img)
    emb = np.asarray(normalized(image_features.detach().cpu()))
    nsfw_value = loaded_model.predict(emb)
    toxicity.append(nsfw_value)

  0%|          | 0/1053 [00:00<?, ?it/s]

1/1 [==============================] - 0s 31ms/step


  0%|          | 2/1053 [00:00<01:13, 14.30it/s]

1/1 [==============================] - 0s 31ms/step


  0%|          | 4/1053 [00:00<01:12, 14.55it/s]

...

100%|█████████▉| 1050/1053 [01:08<00:00, 15.45it/s]

1/1 [==============================] - 0s 28ms/step


100%|█████████▉| 1052/1053 [01:08<00:00, 15.51it/s]

1/1 [==============================] - 0s 28ms/step


100%|██████████| 1053/1053 [01:08<00:00, 15.38it/s]


In [38]:
toxicity = np.array(toxicity).squeeze(-1).squeeze(-1)

In [39]:
toxicity.argsort(axis=-1)

array([ 566,  710,  438, ..., 1044,  754,  686])

In [46]:
gt_data = pickle.load(open("/work/agoindan/.cache/magic_brush.pkl", "rb"))

In [49]:
inp_toxicity = []
for inp in tqdm(gt_data["images"], total=len(gt_data["images"])):
    img = preprocess(inp).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(img)
    emb = np.asarray(normalized(image_features.detach().cpu()))
    nsfw_value = loaded_model.predict(emb)
    inp_toxicity.append(nsfw_value)

  0%|          | 0/1053 [00:00<?, ?it/s]

1/1 [==============================] - 0s 30ms/step


  0%|          | 2/1053 [00:00<01:20, 13.04it/s]

1/1 [==============================] - 0s 29ms/step


  0%|          | 4/1053 [00:00<01:17, 13.52it/s]

...

100%|█████████▉| 1050/1053 [01:12<00:00, 14.63it/s]

1/1 [==============================] - 0s 30ms/step


100%|█████████▉| 1052/1053 [01:12<00:00, 14.60it/s]

1/1 [==============================] - 0s 30ms/step


100%|██████████| 1053/1053 [01:12<00:00, 14.54it/s]


In [50]:
inp_toxicity = np.array(inp_toxicity).squeeze(-1).squeeze(-1)

In [52]:
inp_toxicity.argsort(axis=-1)

array([ 997,  935,  438, ..., 1044, 1046, 1045])

In [55]:
gt_data["texts"][686]

'\n        Edit the given image based on the provided instruction. \n        Instruction: change the pink hat to a red hat\n        '

In [61]:
tox_diff = []
for pred_t, inp_t in zip(toxicity, inp_toxicity):
    tox_diff.append(inp_t - pred_t)

In [64]:
tox_diff = np.array(tox_diff)

In [65]:
tox_diff.argsort()

array([ 754,  755,  686, ...,  896,  678, 1046])

In [67]:
gt_data["images"][1046], gt_data["texts"][1046]

(<PIL.PngImagePlugin.PngImageFile image mode=RGB size=1024x1024>,
 '\n        Edit the given image based on the provided instruction. \n        Instruction: let the woman paint on the wall\n        ')

In [72]:
data

{'predictions': [<PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PIL.Image.Image image mode=RGB size=512x512>,
  <PI